In [ ]:
import numpy as np 
import cvxpy as cp
import matplotlib
import matplotlib.pyplot as plt
import control
from scipy.linalg import solve_discrete_are, solve_discrete_lyapunov, block_diag
import mosek
from scipy.signal import cont2discrete
import pickle

In [ ]:
def sample_m_l_pairs(H):
    m_samples = np.random.uniform(low=.1, high=.5, size=(H, 1))
    l_samples = np.random.uniform(low=.5, high=1, size=(H, 1))
    m_l_pairs = np.hstack((m_samples, l_samples))
    return m_l_pairs

def sample_q_r_pairs(H):
    q_samples = np.random.uniform(low=.1, high=.5, size=(H, 1))
    r_samples = np.random.uniform(low=0.01, high=0.05, size=(H, 1))
    q_r_pairs = np.hstack((q_samples, r_samples))
    return q_r_pairs

def inverted_pendulum(m, l, dt=0.05):
    g = 10
    A = np.array([[1, dt],
                  [(g/l)*dt, 1]])
    B = np.array([[0],
                  [dt/(m*(l**2))]])
    return A, B

def dlqr(A, B, Q, R):
    P = solve_discrete_are(A, B, Q, R)
    K = np.linalg.inv(R + B.T @ P @ B) @ (B.T @ P @ A)
    return K


def squared_grad_heter_bisim_bound(K,Sigma_0,A1,B1,Q1,R1,A2,B2,Q2,R2):
    A1_K = A1-B1@K
    A2_K = A2-B2@K

    assert max(abs(np.linalg.eigvals(A1-B1@K)))<1, "Kn is not a stabilizing controller for system i"
    assert max(abs(np.linalg.eigvals(A2-B2@K)))<1, "Kn is not a stabilizing controller for system j"

    # Find a lambda that retains stability
    A_K = block_diag(A1_K,A2_K)
    lambda_ = np.round(.5*(1-(max(abs(np.linalg.eigvals(A_K))))**2)-1e-5,5)
    
    print("Chosen lambda:",lambda_)
    P1_K = solve_discrete_lyapunov(A1_K.T,Q1+K.T@R1@K)
    P2_K = solve_discrete_lyapunov(A2_K.T,Q2+K.T@R2@K)

    E1_K = (R1+B1.T@P1_K@B1)@K-B1.T@P1_K@A1
    E2_K = (R2+B2.T@P2_K@B2)@K-B2.T@P2_K@A2
    C_K = 2*np.hstack([E1_K,-E2_K])

    # Compute M via optimization problem
    n = Sigma_0.shape[0]
    M = cp.Variable((2*n,2*n),PSD=True)
    M1 = cp.Variable((n,n),PSD=True)
    M2 = cp.Variable((n,n),PSD=True)
    C_K = 2*cp.hstack([E1_K,-E2_K])
    t = cp.Variable()
    s = cp.Variable(nonneg=True)
    u = cp.Variable(nonneg=True)   # epigraph variable for objective

    # Constraints
    constraints = []
    constraints += [M - s * np.eye(2*n) >> 0]    # M >= s I  => s <= lambda_min(M)
    constraints += [t == cp.trace((M1+M2)@Sigma_0)]      # linear relation
    t_tilde = (np.sqrt(2) * t) / (2.0 * lambda_)
    # rotated SOC form: || [2*t_tilde; u - s] ||_2 <= u + s
    constraints += [cp.norm(cp.vstack([2*t_tilde, u - s]), 2) <= u + s]  
    constraints += [s>=1e-6,M>>C_K.T@C_K,A_K.T@M@A_K<<(1-2*lambda_)*M] #M>=1e-5*np.eye(2*n),
    constraints += [M[:n,:n]==M1,M[n:,n:]==M2] #,M[:n,n:] == 0,M[n:,:n] == 0]

    objective = cp.Minimize(u)
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS) 

    if prob.status not in ["infeasible", "unbounded"]:
        M = np.round(M.value)
        M1 = np.round(M1.value,10)
        M2 = np.round(M2.value,10)

        bound = prob.value
        return bound
    else:
        raise ValueError("Problem status: " + prob.status)

def new_squared_grad_heter_bisim_bound(K,Sigma_0,A1,B1,Q1,R1,A2,B2,Q2,R2):
    A1_K = A1-B1@K
    A2_K = A2-B2@K

    assert max(abs(np.linalg.eigvals(A1-B1@K)))<1, "Kn is not a stabilizing controller for system i"
    assert max(abs(np.linalg.eigvals(A2-B2@K)))<1, "Kn is not a stabilizing controller for system j"

    # Find a lambda that retains stability
    A_K = block_diag(A1_K,A2_K)
    lambda_ = np.round(.5*(1-(max(abs(np.linalg.eigvals(A_K))))**2)-1e-5,5)

    print("Chosen lambda:",lambda_)
    P1_K = solve_discrete_lyapunov(A1_K.T,Q1+K.T@R1@K)
    P2_K = solve_discrete_lyapunov(A2_K.T,Q2+K.T@R2@K)

    E1_K = (R1+B1.T@P1_K@B1)@K-B1.T@P1_K@A1
    E2_K = (R2+B2.T@P2_K@B2)@K-B2.T@P2_K@A2
    C_K = 2*np.hstack([E1_K,-E2_K])



    # Compute M via optimization problem
    n = Sigma_0.shape[0]
    M = cp.Variable((2*n,2*n),PSD=True)
    M1 = cp.Variable((n,n),PSD=True)
    M2 = cp.Variable((n,n),PSD=True)
    C_K = 2*cp.hstack([E1_K,-E2_K])
    t = cp.Variable()
    s = cp.Variable(nonneg=True)
    u = cp.Variable(nonneg=True)   # epigraph variable for objective

    # Constraints
    constraints = []
    constraints += [M - s * np.eye(2*n) >> 0]    # M >= s I  => s <= lambda_min(M)
    constraints += [t == cp.trace((M1+M2)@Sigma_0)]      # linear relation
    t_tilde = t / (np.sqrt(2.0) * lambda_)
    # rotated SOC form: || [2*t_tilde; u - s] ||_2 <= u + s
    constraints += [cp.norm(cp.vstack([2*t_tilde, u - s]), 2) <= u + s]  
    constraints += [s>=1e-6,M>>C_K.T@C_K,A_K.T@M@A_K<<(1-2*lambda_)*M] #M>=1e-5*np.eye(2*n),
    constraints += [M[:n,:n]==M1,M[n:,n:]==M2]

    objective = cp.Minimize(u)
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS) 

    if prob.status not in ["infeasible", "unbounded"]:
        M = np.round(M.value)
        M1 = np.round(M1.value,10)
        M2 = np.round(M2.value,10)
        bound = prob.value  
        return bound
    else:
        raise ValueError("Problem status: " + prob.status)

def all_grad_heter_bisim_bounds(K,Sigma_0,As,Bs,Qs,Rs):
    H = len(As)
    result = np.zeros((H,H))
    for i in range(H):
        for j in range(i+1,H):
            print("i,j:",i,j)
            result[i,j] = np.sqrt(squared_grad_heter_bisim_bound(K,Sigma_0,As[i],Bs[i],Qs[i],Rs[i],As[j],Bs[j],Qs[j],Rs[j]))
    return result

# Previous grad-cost heterogeneity matrix
def spectral_norm(M):
    return np.linalg.norm(M, 2)

def max_spectral_norm(mats):
    return max(spectral_norm(M) for M in mats)

def grad_heter_prev_bound(K, As, Bs, Qs, Rs, Sigma_0, epsilons):
    # Compute max cost over all system tuples
    J_max = max(lqr_cost_K(Sigma_0, Ai, Bi, Qi, Ri, K) for Ai, Bi, Qi, Ri in zip(As, Bs, Qs, Rs))

    # Minimum singular value of Q in Qs (smallest among all)
    sigma_min_Qi = min(np.linalg.svd(Qi, compute_uv=False)[-1] for Qi in Qs)

    mu = np.min(np.linalg.eigvalsh(Sigma_0))

    nu = J_max / (sigma_min_Qi * mu)

    # Compute P_K^i for each system i
    PKs = [solve_discrete_lyapunov((Ai - Bi @ K).T, Qi + K.T @ Ri @ K) for Ai, Bi, Qi, Ri in zip(As, Bs, Qs, Rs)]
    norm_PK_max = max_spectral_norm(PKs)  # <-- use max norm over all PKs

    # Max spectral norms over all sets
    norm_A_max = max_spectral_norm(As)
    norm_B_max = max_spectral_norm(Bs)
    norm_Q_max = max_spectral_norm(Qs)
    norm_R_max = max_spectral_norm(Rs)

    norm_A_minus_BK_max = max(
        spectral_norm(Ai - Bi @ K) for Ai, Bi in zip(As, Bs))

    norm_K = spectral_norm(K)
    norm_Sigma_0 = spectral_norm(Sigma_0)

    # h1_het(K)
    h1 = (J_max / sigma_min_Qi) * norm_B_max * (
        norm_PK_max + 4 * norm_A_minus_BK_max**2 * (norm_Q_max + norm_K**2 * norm_R_max) * nu**2)
    h1 += 4 * (norm_R_max * norm_K +
               (J_max * (norm_B_max**2 * norm_K + norm_B_max * norm_A_max)) / mu) * nu**2 * norm_A_minus_BK_max * norm_Sigma_0

    # h2_het(K)
    h2 = (J_max / sigma_min_Qi) * (
        norm_A_minus_BK_max * norm_PK_max + norm_B_max * norm_PK_max * norm_K
    + 4 * norm_K * norm_A_minus_BK_max**2 * (norm_Q_max + norm_K**2 * norm_R_max) * nu**2)
    h2 += 4 * (norm_R_max * norm_K +
               (J_max * (norm_B_max**2 * norm_K + norm_B_max * norm_A_max)) / mu) * nu**2 * norm_K * norm_A_minus_BK_max * norm_Sigma_0

    # h3_het(K)
    h3 = (J_max / sigma_min_Qi) * norm_B_max * norm_A_minus_BK_max * nu

    # h4_het(K)
    h4 = (J_max / sigma_min_Qi) * (norm_K + norm_K**2 * norm_B_max * norm_A_minus_BK_max * nu)

    eps1, eps2, eps3, eps4 = epsilons
    dx = As[0].shape[0]
    du = Bs[0].shape[1]
    bound = np.sqrt(min(dx,du)) * (eps1 * h1 + eps2 * h2 + eps3 * h3 + eps4 * h4)
    return bound

#Multitask-LQR (for negative feedback u=-Kx)
def grad_true(Sigma_0,A,B,Q,R,K):
    dx = A.shape[0]
    PK = solve_discrete_lyapunov((A-B@K).T,Q+K.T@R@K)
    EK = (R + B.T@PK@B)@K - B.T@PK@A
    Sigma_K = solve_discrete_lyapunov(A-B@K,Sigma_0)
    grad = 2*EK@Sigma_K
    return grad

def lqr_cost_K(Sigma_0,A,B,Q,R,K):
        PK = solve_discrete_lyapunov((A-B@K).T,Q+K.T@R@K)
        return np.trace(Sigma_0@PK)

def lqr_cost_avg_K(Sigma_0,As,Bs,Qs,Rs,K):
        H = len(As)
        cost_avg = 0
        for i in range(H):
            cost_i = lqr_cost_K(Sigma_0,As[i],Bs[i],Qs[i],Rs[i],K)
            cost_avg += (1/H)*cost_i
        return cost_avg

def grad_avg(As,Bs,Qs,Rs,K):
    dx = As[0].shape[0]
    du = Bs[0].shape[1]
    H = len(As)
    grad_avg = np.zeros((du,dx))
    for i in range(H):
        grad_cost_i = grad_true(Sigma_0,As[i],Bs[i],Qs[i],Rs[i],K)
        grad_avg += (1/H)*grad_cost_i
    return grad_avg

def max_offdiag_norm(matrices):
    N = len(matrices)
    max_norm = 0.0
    for i in range(N):
        for j in range(i+1, N):
            diff_norm = np.linalg.norm(matrices[i] - matrices[j], ord=2)
            if diff_norm > max_norm:
                max_norm = diff_norm
    return max_norm

def compute_gener_error_bound(bisimbound, Jmax, Sigma_0, Rs, Qs):
    # Minimum eigenvalue of Sigma_0
    lambda_min = np.min(np.linalg.eigvals(Sigma_0)).real
    
    # Minimum singular value across all Ri
    sigma_min_R = min(np.min(np.linalg.svd(Ri, compute_uv=False)) for Ri in Rs)
    
    # Minimum singular value across all Qi
    sigma_min_Q = min(np.min(np.linalg.svd(Qi, compute_uv=False)) for Qi in Qs)
    
    # Compute the bound
    bound = (2 * bisimbound**2 * Jmax) / (lambda_min**2 * sigma_min_R * sigma_min_Q)
    return bound

def multi_task_LQR(Js_star, Sigma_0, K0, eta, N, As, Bs, Qs, Rs):
    K = K0
    H = len(As)
    Ks = []
    grad_heter = []
    bisim_grad_heter = []
    prev_grad_heter = []
    Js_Ks = np.zeros((H,N))
    gener_error_bounds = []
    costs_avg = []
    bisim_bounds = np.zeros((H,N))

    eps1 = max_offdiag_norm(As)
    eps2 = max_offdiag_norm(Bs)
    eps3 = max_offdiag_norm(Qs)
    eps4 = max_offdiag_norm(Rs)
    epsilons = (eps1, eps2, eps3, eps4)

    x_vals = np.arange(N)  # fixed x-axis

    for n in range(N):
        Ks.append(K)
        cost_avg = lqr_cost_avg_K(Sigma_0, As, Bs, Qs, Rs, K)
        costs_avg.append(cost_avg)
        grad_average = grad_avg(As, Bs, Qs, Rs, K)
        for i in range(H):
            Js_Ks[i,n] = lqr_cost_K(Sigma_0, As[i], Bs[i], Qs[i], Rs[i], K)  

        grads_true = np.stack([grad_true(Sigma_0, As[i], Bs[i], Qs[i], Rs[i], K) for i in range(H)])
        diffs = grads_true[:, None, :, :] - grads_true[None, :, :, :]
        grad_heter.append(np.max(np.linalg.norm(diffs, axis=(2, 3))))
        bisim_matrix = all_grad_heter_bisim_bounds(K, Sigma_0, As, Bs, Qs, Rs)
        bisimbound = (1/H)*np.max(np.sum(bisim_matrix+bisim_matrix.T, axis=0))
        print("bisim_matrix:",bisim_matrix)
        bisim_grad_heter.append(bisimbound)
        prev_grad_heter.append(grad_heter_prev_bound(K, As, Bs, Qs, Rs, Sigma_0, epsilons))
        gener_error_bounds.append(compute_gener_error_bound(bisimbound, max(Js_Ks[:,n]), Sigma_0, Rs, Qs))

        K = K - eta * grad_average

        print(f"Iteration: {n+1}, Avg cost: {cost_avg}")
        print(f"Grad. heter.: {grad_heter[-1]}")
        print(f"Bisim. grad. heter.: {bisim_grad_heter[-1]}")
        print(f"Prev. grad. heter.: {prev_grad_heter[-1]}")
        print("---------")

        if n == N-1:
            max_optim_err = np.max(Js_Ks[i,n]-Js_star[i])
    return grad_heter, bisim_grad_heter, prev_grad_heter, bisim_bounds, max_optim_err, Js_Ks, bisim_bounds

In [ ]:
# Sample H tasks
H = 6
m_l_pairs = sample_m_l_pairs(H)
q_r_pairs = sample_q_r_pairs(H)

As = [inverted_pendulum(m, l)[0] for m, l in m_l_pairs]
Bs = [inverted_pendulum(m, l)[1] for m, l in m_l_pairs]
Qs = [q*np.eye(2) for q, r in q_r_pairs]
Rs = [r*np.eye(1) for q, r in q_r_pairs]

avg_sr = 0
for i in range(H):
    K = dlqr(As[i], Bs[i], Qs[i], Rs[i])
    Acl = As[i] - Bs[i] @ K
    sr = max(abs(np.linalg.eigvals(Acl)))
    avg_sr += sr
    print(f"System {i+1}: spectral radius = {sr:.4f}")

In [ ]:
dx = As[0].shape[0]
Sigma_0 = 0.1*np.eye(dx)
eta = 1e-2
idx = 0
K0 = 0
for idx in range(H):
    Kopt, _, _ = control.dlqr(As[idx], Bs[idx], Qs[idx], Rs[idx])
    K0 += Kopt + 1 * np.random.randn(1,2)
K0 *= 1/H

Js_star = []
for i in range(len(As)):
    assert max(abs(np.linalg.eigvals(As[i] - Bs[i] @ K0))) < 1, "K0 is not a common stabilizing controller"
    _, Pi_star, _ = control.dlqr(As[i], Bs[i], Qs[i], Rs[i])
    Js_star.append(np.trace(Sigma_0@Pi_star))

    
N = 1000
eps1 = max_offdiag_norm(As)
eps2 = max_offdiag_norm(Bs)
eps3 = max_offdiag_norm(Qs)
eps4 = max_offdiag_norm(Rs)
epsilons = (eps1, eps2, eps3, eps4)
print("epsilons:",epsilons)
grad_heter, bisim_grad_heter, prev_grad_heter, bisim_bounds, max_optim_err, Js_Ks, bisim_bounds = multi_task_LQR(Js_star, Sigma_0, K0, eta, N, As, Bs, Qs, Rs)
print("epsilons:",epsilons)
print("Maximum epsilon:",max(epsilons))
print("prev_grad_heter_infty:",prev_grad_heter[-1])
print("d_infty:", np.max(bisim_bounds[:,-1]))
print("max_task_specific_optim_error_infty:",max_optim_err)